In [ ]:
#HCA_ndar_edinburgh_2022release
#(only thing that changed was the Rosetta file pointer)
#adding event to ndar_subjects (element name is 'visit') - Nov 10, 2020
#pointing to new Rosetta again

In [ ]:
import os, datetime
import csv
import sys
import shutil
from openpyxl import load_workbook
import pandas as pd
from io import BytesIO
import numpy as np
import subprocess
from scipy import stats
from ccf.box import LifespanBox
from ccf.config import LoadSettings
from ccf.redcap import RedcapTable 
config = LoadSettings()
eventlist=['V1','V2']

In [ ]:
box=LifespanBox(cache='./')
verbose = True
snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')
pathout="./prepped/hca" 

#Rosetta (a.k.a Inventory) file will have all the nda vars and pedids
extrainfo=config['rosetta']['filename']

redcapconfigfile="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/.boxApp/redcapconfig.csv"

snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')
#catfromdate=max of last run--'2019-06-17'
snapshotdate

In [ ]:
#defer to rosetta for dates but check - see email 11/10/2020 for GUIDs that were affected by mismatch between IntraDB and Redcap
#extract the inventory subset 
inventory=pd.read_csv(extrainfo)[['DB_Source','REDCap_id','subject','pseudo_guid','nda_interview_date','nda_age','M/F','pedid','redcap_event_name','redcap_event','race','ethnic_group','site']]
inventory=inventory.loc[inventory.nda_age.isnull()==False]
inventory.nda_interview_date=pd.to_datetime(inventory.nda_interview_date).dt.strftime('%m/%d/%Y')
inventory.nda_age=inventory.nda_age.round(0).astype(int)
print('Inventory Shape out of the Box',inventory.shape)
print('Number of Unique Subjects',len(inventory.subject.unique()))
#rosetta=pd.read_csv(hcplist,header=0)[['nda_guid','subjectped','nda_interview_age','nda_interview_date','nda_gender','final_pedid']]
#rosetta.nda_interview_date=pd.to_datetime(rosetta.nda_interview_date).dt.strftime('%m/%d/%Y')
#rosetta.head()

# Prepare Ped File

In [ ]:
#curated pedigrees contain redundant information with inventory (e.g. pedids and pseudoguids, HCA_ID, HCD_ID, etc).
peds=883930709978
pathpeds=box.downloadFile(peds)
pedids=pd.read_csv(pathpeds)
pedids=pedids.drop(columns=['psuedo_guid','Familyflag','site','parent','age','sex','relationship1', 'with1', 'relationship2', 'with2', 'relationship3',
       'with3', 'relationship4', 'with4', 'relationship5', 'with5', 'src_avuncular1_id','src_grandparent1_id','src_cousin_id1','src_cousin_id2',
       'relationship6', 'with6', 'relationship7', 'with7'])
pedids.loc[pedids.subject.str.contains('HCA')].head()
print('shape before:',pedids.loc[pedids.subject.str.contains('HCA')].shape)

#pedids.loc[(pedids.subject.str.contains('HCA')) | (pedids.HCA_ID.str.contains('HCA'))]
pedids.loc[(pedids.HCA_ID.isnull()==False),'subject']=pedids.HCA_ID
print('shape after:',pedids.loc[pedids.subject.str.contains('HCA')].shape)
print(inventory.columns)
print(pedids.columns)
pedids=pedids.loc[pedids.subject.str.contains('HCA')]


In [ ]:
inventory2=pd.merge(inventory.drop(columns=['pedid']),pedids,on='subject',how='outer',indicator=True)
inventory2['visit']=inventory2.redcap_event
inventory2._merge.value_counts()
ndarinventory=inventory2.copy()

In [ ]:
ndarinventory['interview_date']=pd.to_datetime(ndarinventory.nda_interview_date).dt.strftime('%m/%d/%Y')
ndarinventory['interview_age']=ndarinventory.nda_age.round(0).astype(int)
siblist=['sibling_type1','sibling_type2','sibling_type3']
for ii in siblist:
    ndarinventory[ii]=ndarinventory.replace({ii:
                                       {'FB':'Full Brother FB',
                                        'HMB':'Half Mother Brother HMB',
                                        'HFB':'Half Father Brother HFB',
                                        'FS':'Full Sister FS',
                                        'HMS':'Half Mother Sister HMS',
                                        'HFS':'Half Father Sister HFS',
                                        'AB':'Adopted Brother AB',
                                        'AS':'Adopted Sister AS',
                                       }})[ii]


In [ ]:

ndarinventory.loc[:,'phenotype']=pd.Series(("Healthy Subject" for i in range(ndarinventory.shape[0])),index=ndarinventory.index)
ndarinventory.loc[:,'phenotype_description']=pd.Series(("No diagnosed history of neurologic or major psychiatric disorder" for i in range(ndarinventory.shape[0])),index=ndarinventory.index)
ndarinventory.loc[:,'twins_study']=pd.Series(("No" for i in range(ndarinventory.shape[0])),index=ndarinventory.index)
ndarinventory.loc[:,'sibling_study']=pd.Series(("No" for i in range(ndarinventory.shape[0])),index=ndarinventory.index)
ndarinventory.loc[:,'family_study']=pd.Series(("No" for i in range(ndarinventory.shape[0])),index=ndarinventory.index)
ndarinventory.loc[:,'sample_taken']=pd.Series(("No" for i in range(ndarinventory.shape[0])),index=ndarinventory.index)


In [ ]:

#capture missing race
flagsz=pd.DataFrame(ndarinventory.loc[ndarinventory.race==''])
flagsz

#flags=flagsz[['subject','site','interview_date']].copy()
##flags.loc[:,'reason']=pd.Series(("No race" for i in range(flags.shape[0])),index=flags.index)
#flags

In [ ]:
#capture missing ethnicity
flagsez=pd.DataFrame(ndarinventory.loc[ndarinventory.ethnic_group==''])
flagsez

#flagsez=pd.DataFrame(dfnew.loc[dfnew.ethnic_group==''])
#flagse=flagsez[['subject_id','site_char','nda_interview_date']].copy()
#flagse.loc[:,'reason']=pd.Series(("No ethnicity" for i in range(flagse.shape[0])),index=flagse.index)


In [ ]:
ndarinventory.columns
#ndarinventory.head()
ndarinventory.visit.value_counts()

In [ ]:
#next for external use uploading to NDA
#remove unneccessary vars                                                                   and rename - note cant rename genderstring as gender until original numeric gender var is removed
ndar=ndarinventory.drop(columns=['sibling_type1_alt','HCD_ID', 'HCA_ID','DB_Source', 'REDCap_id',
                                   'redcap_event_name','adoptmomsubj', 'adoptdadsubj','_merge','nda_age','redcap_event','nda_interview_date'])
ndar=ndar.rename(columns={"subject":"src_subject_id",'M/F':"sex",
                           'biomomsubj':'src_mother_id', 'biodadsubj':'src_father_id',
                           "pedid":"family_user_def_id","nda_guid":"subjectkey",
                           "site_char":"site",'pseudo_guid':'subjectkey'})

print(ndar.shape)
ndar=ndar.loc[ndar.visit.isin(eventlist)].copy()
ndar.head()


In [ ]:
#write out csv for validation
filePath=pathout+'/ndar_subject01.csv'

if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("ndar_subject,1\n")
    ndar.to_csv(f,index=False)



In [ ]:
fieldlist=['redcap_event_name','id','psuedo_guid','iihandwr','iihandth','iihandsc','iihandto','iihandkn','iihandsp','iihandbr','iihandma','iihandbo','iihandfk','iihandey']
hca=RedcapTable.get_table_by_name('hcpa').get_frame(fieldlist)
#hca=box.getredcapfields(fieldlist, study='hcpa') #this function autmatically gets interview_date and interview_age...but want to defer to rosetta for these fields
#hca=hca.drop(columns=['interview_date','interview_age','psuedo_guid','gender','dob']).copy()
hca.columns

In [ ]:
forhands=pd.merge(ndarinventory.drop(columns=['_merge']),hca,left_on=['REDCap_id','redcap_event_name'],right_on=['id','redcap_event_name'],how='left')#,indicator=True)
#forhands._merge.value_counts()   
forhands.columns
forhands2=forhands.rename(columns={"subject":"src_subject_id",'M/F':"gender",'pseudo_guid':'subjectkey'})
forhands2=forhands2.loc[forhands2.redcap_event.isin(eventlist)]

In [ ]:

hand=forhands2[['redcap_event','src_subject_id','gender', 'subjectkey', 'interview_date', 'interview_age','iihandwr','iihandth','iihandsc','iihandto', 'iihandkn','iihandsp', 'iihandbr', 'iihandma','iihandbo','iihandfk','iihandey',]].copy()
varlist=["iihandwr","iihandth","iihandsc","iihandto","iihandkn","iihandsp","iihandbr","iihandma","iihandbo","iihandfk","iihandey"]
varlistnoeye=["iihandwr","iihandth","iihandsc","iihandto","iihandkn","iihandsp","iihandbr","iihandma","iihandbo","iihandfk"]
hand.to_csv('findmissings.csv')

In [ ]:
#assign these to median of their row for sum
hand.loc[(hand.src_subject_id=='HCA7319472') & (hand.interview_age==788) ,'iihandma']=1
hand.loc[(hand.src_subject_id=='HCA7605978') & (hand.interview_age==474) ,'iihandfk']=5
hand.loc[(hand.src_subject_id=='HCA8324975') & (hand.interview_age==1005),'iihandma']=5
hand.loc[(hand.src_subject_id=='HCA6476584') & (hand.interview_age==889) ,'iihandbo']=5                                            

    
#convert to ints for sums
for var in varlist:
    hand[var]=hand[var].astype(float)

hand.loc[hand.src_subject_id.isin(['HCA6476584','HCA8324975','HCA7605978','HCA7319472'])] 


In [ ]:
hand['handvarsum']=hand[varlistnoeye].sum(axis=1)
hand.loc[:,'handvarconstant']=pd.Series((3*len(varlistnoeye) for i in range(hand.shape[0])),index=hand.index)
hand['hcp_handedness_score']=5*(hand['handvarsum']-hand['handvarconstant'])
hand=hand.drop(['handvarsum','handvarconstant'],axis=1)

hand.dtypes

In [ ]:
#assign these back to -99 Missing- (not possible to track down), and convert to string below
hand.loc[(hand.src_subject_id=='HCA7319472') & (hand.interview_age==788) ,'iihandma']=-99
hand.loc[(hand.src_subject_id=='HCA7605978') & (hand.interview_age==474) ,'iihandfk']=-99
hand.loc[(hand.src_subject_id=='HCA8324975') & (hand.interview_age==1005),'iihandma']=-99
hand.loc[(hand.src_subject_id=='HCA6476584') & (hand.interview_age==889) ,'iihandbo']=-99   

#drop these two who have no data
print(hand.shape)
hand=hand.loc[~((hand.src_subject_id=='HCA9155680') & (hand.interview_age== 1015))]
hand=hand.loc[~((hand.src_subject_id=='HCA8249078') & (hand.interview_age== 734))]
print(hand.shape)
#HCA8249078 734

#hand.dtypes
#hand.loc[hand.src_subject_id.isin(['HCA6476584','HCA8324975','HCA7605978','HCA7319472'])] 
#hand.to_csv('wtf.csv')
for ii in varlist:
     hand[ii]=hand[ii].fillna(-99).round().astype('int').astype(str)
     hand.loc[hand[ii]=='-99',ii]=''

#    #hand.loc[hand[varlist[ii]]=='-99',varlist[ii]]=''
#    print(hand[ii].dtypes)

#for ii in range(len(varlist)):
#    hand.loc[varlist[ii]]=hand[varlist[ii]].fillna(0).astype('int')
#    print(hand[varlist[ii]].fillna(-99).value_counts())
#for ii in range(len(varlist)):
#    hand.loc[hand[varlist[ii]]=='-99',varlist[ii]]=''
#
##hand.loc[hand.src_subject_id.isin(['HCA6476584','HCA8324975','HCA7605978','HCA7319472'])] 
#hand.dtypes

In [ ]:
#translate to nda values (e.g. #spr;right;spl;left;both;np;ne; oh
#SPR=Strongly Prefer Right; SPL=Strongly Prefer Left; ne =not experienced; np=no preference; oh=sometimes uses other hand
iihandlist=['iihandwr','iihandth','iihandsc','iihandto', 'iihandkn','iihandsp', 'iihandbr', 'iihandma','iihandbo','iihandfk','iihandey']
newhandlist=['newiihandwr','newiihandth','newiihandsc','newiihandto', 'newiihandkn','newiihandsp', 'newiihandbr', 'newiihandma','newiihandbo','newiihandfk','newiihandey']
ndahandlist=['writing','throwing','scissors','toothbrush', 'knife_no_fork','spoon', 'broom', 'match','box','foot','eye']

for ii in range(len(iihandlist)):
    hand[newhandlist[ii]]=hand.replace({iihandlist[ii]:
                                       {'1':'left',
                                        '2':'spl',
                                        '3':'np',
                                        '4':'spr',
                                        '5':'right'}})[iihandlist[ii]]



In [ ]:
hand=hand.drop(columns=iihandlist,axis=1)
dictnames=dict(zip(newhandlist,iihandlist))
hand=hand.rename(columns=dictnames)


In [ ]:
hand.columns
#rename them all to nda elements
tondanames=dict(zip(iihandlist,ndahandlist))
hand=hand.rename(columns=tondanames)
hand.columns
hand=hand.drop(columns=['redcap_event'])
hand.hcp_handedness_score=hand.hcp_handedness_score.astype(int)
hand=hand.rename(columns={'gender':'sex'})

In [ ]:

filePath=pathout+'/edinburgh_hand01.csv'
if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("edinburgh_hand,1\n")
    hand.to_csv(f,index=False)
